<img src="imagenes/rn3.png" width="200">
<img src="http://www.identidadbuho.uson.mx/assets/letragrama-rgb-150.jpg" width="200">

# [Curso de Redes Neuronales](https://curso-redes-neuronales-unison.github.io/Temario/)

# Una red neuronal multicapa simple usando TensorFlow


[**Julio Waissman Vilanova**](http://mat.uson.mx/~juliowaissman/), 27 de septiembre de 2017.



En esta libreta se muestra el ejemplo básico para una red multicapa sencilla
aplicada al conjunto de datos [MNIST](http://yann.lecun.com/exdb/mnist/).

Esta libreta es básicamente una traducción del ejemplo
desarrollado por [Aymeric Damien](https://github.com/aymericdamien/TensorFlow-Examples/)


In [3]:
import tensorflow as tf

## 1. Cargar datos

Primero cargamos los archivos que se utilizan para el aprendizaje. Para otro tipo de problemas, es necesario hacer un proceso conocido como *Data Wrangling*, que normalmente se realiza con la ayuda de *Pandas*. 

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from t

Para que un aprendizaje tenga sentido es necesario tener bien separado un conjunto de datos de aprendizaje y otro de prueba (en caso de grandes conjuntos de datos es la opción). Como vemos tanto las imágenes como las etiquetas están separados en archivos de datos y de aprendizaje.

El objeto `mnist` es un objeto tensorflow que contiene 3 objetos tipo tensorflow: *test*, *train* y *validation*, los cuales a su vez contienen *ndarrays* de *numpy*. La estructura es la misma para cada conjunto de datos. Veamos su estructura:


In [5]:
print("Tipo de images: {}".format(type(mnist.train.images)))
print("Tipo de epochs_completed: {}".format(type(mnist.train.epochs_completed)))
print("Tipo de labels: {}".format(type(mnist.train.labels)))
print("Tipo de nest_batch: {}".format(type(mnist.train.next_batch)))
print("Tipo de num_examples: {}".format(type(mnist.train.num_examples)))

Tipo de images: <class 'numpy.ndarray'>
Tipo de epochs_completed: <class 'int'>
Tipo de labels: <class 'numpy.ndarray'>
Tipo de nest_batch: <class 'method'>
Tipo de num_examples: <class 'int'>


Como generar el conjunto de datos para ser utilizado dentro de TensorFlow es objeto de otra libreta. Por el momento concentremonos en como hacer una red neuronal rápido y sin dolor.

Sin embargo, vamos a ver unos cuantos datos que nos pueden ser de útilidad para la construcción de la red neuronal.

In [6]:
print("Forma del ndarray con las imágenes: {}".format(mnist.train.images.shape))
print("Forma del ndarray con las etiquetas: {}".format(mnist.train.labels.shape))
print("-" * 79)
print("Número de imagenes de entrenamiento: {}".format(mnist.train.images.shape[0]))
print("Tamaño de las imagenes: {}".format(mnist.train.images.shape[1]))
print("Clases diferentes: {}".format(mnist.train.labels.shape[1]))


Forma del ndarray con las imágenes: (55000, 784)
Forma del ndarray con las etiquetas: (55000, 10)
-------------------------------------------------------------------------------
Número de imagenes de entrenamiento: 55000
Tamaño de las imagenes: 784
Clases diferentes: 10


## 2. Construcción de la red neuronal

Para hacer una red neuronal lo más genérica posible y que pdamos reutilizar en otros proyectos, vamos a establecer los parámetros base independientemente de la inicialización de la red, independientemente de la forma en que construimos la red. 

Comencemos por establecer una función genérica que nos forme una red neuronal con dos capas ocultas. No agrego más comentarios porque, con la experiencia de las libretas anteriores, la construcción de la red neuronal se explica sola.

In [7]:
def red_neuronal_dos_capas_ocultas(x, pesos, sesgos):
    """
    Genera una red neuronal de dos capas para usar en TensorFlow
    
    Parámetros
    ----------
    pesos: un diccionario con tres etiquetas: 'h1', 'h2' y 'ho'
           en donde cada una es una tf.Variable conteniendo una 
           matriz de dimensión [num_neuronas_capa_anterior, num_neuronas_capa]
                  
    sesgos: un diccionario con tres etiquetas: 'b1', 'b2' y 'bo'
            en donde cada una es una tf.Variable conteniendo un
            vector de dimensión [numero_de_neuronas_capa]
                   
    Devuelve
    --------
    Un ops de tensorflow que calcula la salida de una red neuronal
    con dos capas ocultas, y activaciones RELU.
    
    """
    # Primera capa oculta con activación ReLU
    capa_1 = tf.matmul(x, pesos['h1'])
    capa_1 = tf.add(capa_1, sesgos['b1'])
    capa_1 = tf.nn.relu(capa_1)
    
    # Segunda capa oculta con activación ReLU
    capa_2 = tf.matmul(capa_1, pesos['h2'])
    capa_2 = tf.add(capa_2, sesgos['b2'])
    capa_2 = tf.nn.relu(capa_2)
    
    # Capa de salida con activación lineal
    # En Tensorflow la salida es siempre lineal, y luego se especifica
    # la función de salida a la hora de calcularla como vamos a ver 
    # más adelante
    capa_salida = tf.matmul(capa_2, pesos['ho']) + sesgos['bo']
    return capa_salida

Y ahora necesitamos poder generar los datos de entrada a la red neuronal de
alguna manera posible. Afortunadamente sabemos exactamente que necesitaos, así
que vamos a hacer una función que nos genere las variables de peso y sesgo.

Por el momento, y muy a la brava, solo vamos a generarlas con números aletorios con una 
distribución $\mathcal{N}(0, 1)$.

In [8]:
def inicializa_pesos(entradas, n1, n2, salidas):
    """
    Genera un diccionario con pesos  
    para ser utilizado en la función red_neuronal_dos_capas_ocultas
    
    Parámetros
    ----------
    entradas: Número de neuronas en la capa de entrada
    
    n1: Número de neuronas en la primer capa oculta
    
    n2: Número de neuronas en la segunda capa oculta
    
    salidas: Número de neuronas de salida
    
    Devuelve
    --------
    Dos diccionarios, uno con los pesos por capa y otro con los sesgos por capa
    
    """
    pesos = {
        'h1': tf.Variable(tf.random_normal([entradas, n1])),
        'h2': tf.Variable(tf.random_normal([n1, n2])),
        'ho': tf.Variable(tf.random_normal([n2, salidas]))
    }
    
    sesgos = {
        'b1': tf.Variable(tf.random_normal([n1])),
        'b2': tf.Variable(tf.random_normal([n2])),
        'bo': tf.Variable(tf.random_normal([salidas]))
    }
    
    return pesos, sesgos

Ahora necesitamos establecer los parámetros de la topología de la red neuronal. 
Tomemos en cuenta que estos prámetros los podríamos haber establecido desde
la primer celda, si el fin es estar variando los parámetros para escoger los que 
ofrezcan mejor desempeño.

In [9]:
num_entradas = 784  #  Lo sabemos por la inspección que hicimos a mnist
num_salidas = 10    # Ídem

# Aqui es donde podemos jugar
num_neuronas_capa_1 = 256
num_neuronas_capa_2 = 256

¡A construir la red! Para esto vamos a necesitar crear las entradas
con un placeholder, y crear nuestra topología de red neuronal.

Observa que la dimensión de x será [None, num_entradas], lo que significa que 
la cantidad de renglones es desconocida (o variable).

In [10]:
# La entrada a la red neuronal
x = tf.placeholder("float", [None, num_entradas])

# Los pesos y los sesgos
w, b = inicializa_pesos(num_entradas, num_neuronas_capa_1, num_neuronas_capa_2, num_salidas)

# Crea la red neuronal
estimado = red_neuronal_dos_capas_ocultas(x, w, b)


Instructions for updating:
Colocations handled automatically by placer.


Parecería que ya está todo listo. Sin ambargo falta algo muy importante: No hemos explicado
ni cual es el criterio de error (loss) que vamos a utilizar, ni cual va a ser el método de
optimización (aprendizaje) que hemos decidido aplicar.

Primero definamos el costo que queremos minimizar, y ese costo va a estar en función de lo
estimado con lo real, por lo que necesitamos otra entrada de datos para los datos de salida.

Sin ningun lugar a dudas, el costo que mejor describe este problema es el de *softmax*


In [11]:
#  Creamos la variable de datos de salida conocidos
y = tf.placeholder("float", [None, num_salidas])

#  Definimos la función de costo
# softmax con crossentropy con logits esta por ser obsoleto: actualizar a la v2
costo = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=estimado, labels=y))

Y ahora definimos que función de aprendizaje vamos a utilizar. Existen muchas funciones
de aprendizaje en tensorflow, las cuales se pueden consultar en `tf.train.`. Entre las
existentes podemos ver algunas conocidas del curso como descenso de gradiente simple,
momento, rprop, rmsprop entre otras. Casi todas las funciones de optimización (aprendizaje)
acaban su nombre con `Optimize`.

En este caso vamos a usar un método comocido como el *algoritmo de Adam* el cual 
se puede consultar [aqui](http://arxiv.org/pdf/1412.6980.pdf). El metodo utiliza dos calculos
de momentos diferentes, y por lo visto genera resultados muy interesantes desde el punto 
de vista práctico.

¿Cual es el mejor método? Pues esto es en función de tu problema y de la cantidad de datos que tengas.
Lo mejor es practicar con varios métodos para entender sus ventajas y desventajas.

En todo caso el método de optimización requiere que se le inicialice con una tasa de aprendizaje.

In [12]:
alfa = 0.001
optimizador = tf.train.AdamOptimizer(learning_rate=alfa)
paso_entrenamiento = optimizador.minimize(costo)


## 3. Ejecutar la sesión usando mini-batches

Ahora, ya que la red neuronal está lista vamos a ejecutar la red utilizando el algoritmo de
Adam pero en forma de mini-batches. Con el fin de tener control sobre el problema, vamos a establecer un número máximo de epoch (ciclos de aprendizaje), el tamaño de los mini-batches, y cada cuandos epoch 
quisieramos ver como está evolucionando la red neuronal.

Como entrenar una red neuronal no tiene sentido, si no es porque la queremos usar para reconocer,
no tendría sentido entrenarla y luego perderla y tener que reentrenar en cada ocasión. Recuerda que cuando
se cierra la sesión se borra todo lo que se tenía en memoria. 

Para esto vamos a usar una ops especial llamada `Saver`, que permite guardar en un archivo la red neuronal y 
después utilizarla en otra sesión (en otro script, computadora, ....).


In [13]:
archivo_modelo = "rnn2.ckpt"
saver = tf.train.Saver()

Como todo se ejecuta dentro de una sesión, no es posible hacerlo por partes (si usamos el 
`with` que debería ser la única forma en la que iniciaramos una sesión). Por lo tanto procuraré dejar comentado el código.

In [14]:
numero_epochs = 30
tamano_minibatch = 100
display_step = 1

# Muy importante la primera vez que se ejecuta inicializar todas las variables
init = tf.global_variables_initializer()

# La manera correcta de iniciar una sesión y realizar calculos
with tf.Session() as sess:
    sess.run(init)

    # Ciclos de entrenamiento
    for epoch in range(numero_epochs):

        #  Inicializa el costo promedio de todos los minibatches en 0
        avg_cost = 0.
        
        #  Calcula el número de minibatches que se pueden usar 
        total_batch = int(mnist.train.num_examples/tamano_minibatch)

        #  Por cada minibatch
        for i in range(total_batch):
            
            #  Utiliza un generador incluido en mnist que obtiene 
            #  tamano_minibatch ejemplos selecionados aleatoriamente del total
            batch_x, batch_y = mnist.train.next_batch(tamano_minibatch)
            
            #  Ejecuta la ops del paso_entrenamiento para aprender 
            #  y la del costo, con el fin de mostrar el aprendizaje
            _, c = sess.run([paso_entrenamiento, costo], feed_dict={x: batch_x, y: batch_y})
            
            #  Calcula el costo del minibatch y lo agrega al costo total
            avg_cost += c / total_batch
        
        # Muestra los resultados
        if epoch % display_step == 0:
            print (("Epoch: " + str(epoch)).ljust(20)
                   + ("Costo: " + str(avg_cost)))
    
    #  Guarda la sesión en el archivo rnn2.cptk
    saver.save(sess, archivo_modelo)
    
    print("Se acabaron los epochs, saliendo de la sesión de tensorflow.")

Epoch: 0            Costo: 191.18521693489762
Epoch: 1            Costo: 43.71823168321089
Epoch: 2            Costo: 26.89267680254849
Epoch: 3            Costo: 18.644550804443778
Epoch: 4            Costo: 13.412553062162813
Epoch: 5            Costo: 9.914526332187547
Epoch: 6            Costo: 7.315131008469698
Epoch: 7            Costo: 5.454284337533445
Epoch: 8            Costo: 4.0039133195275065
Epoch: 9            Costo: 3.00410714654874
Epoch: 10           Costo: 2.2139289501789805
Epoch: 11           Costo: 1.6914308566438865
Epoch: 12           Costo: 1.220751516974679
Epoch: 13           Costo: 0.961380459997427
Epoch: 14           Costo: 0.7873306533359897
Epoch: 15           Costo: 0.6912819738954676
Epoch: 16           Costo: 0.5229584433662885
Epoch: 17           Costo: 0.6364421827525495
Epoch: 18           Costo: 0.3839469246409922
Epoch: 19           Costo: 0.4402724905136021
Epoch: 20           Costo: 0.38588204460730374
Epoch: 21           Costo: 0.3756406374220

Ahora vamos a revisar que tan bien realizó el aprendizaje cuando se aplica la red adatos que
no se usaron para entrenamiento. Para esto vamos a utilizar dos ops extas: una 
para definir la operaración de datos bien estimados o mal estimados, y otra para
calcular el promedio de datos bien estimados. Para calcular los datos bien estimados vamos a utilizar `tf.cast` que permite ajustar los tipos
al tipo tensor. 

Recuerda que, al haber definido `prediccion_correcta` y `precision` como ops de tensorflow *después* de la ultima vez que usaste dentro de una sesión la ops `saver`, estas operaciones no se guardaron en memoria. Si quieres conservarlas, debes de definirlas *antes* de ejecutar el aprendizaje.

In [15]:
prediction_correcta = tf.equal(tf.argmax(estimado, 1), tf.argmax(y, 1))

precision = tf.reduce_mean(tf.cast(prediction_correcta, "float"))



Ahora si, vamos a abrir una nueva sesión, vamos a restaurar los valores de la sesión anterior,
y vamos a ejecutar el grafo con el fin de evaluar la ops precision, pero ahora con el
diccionario de alimentación con los datos de prueba.

In [16]:
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, archivo_modelo)
    porcentaje_acierto = sess.run(precision, feed_dict={x: mnist.test.images,
                                                        y: mnist.test.labels})
    print("Precisión: {}".format(porcentaje_acierto))


Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from rnn2.ckpt
Precisión: 0.9526000022888184


## 4. Contesta las siguientes preguntas

1. ¿Que pasa si aumenta el número de epochs? ¿Cuando deja de ser util aumentar los epoch?

2. ¿Que pasa si aumentas o disminuyes la tasa de aprendizaje?

3. Utiliza al menos otros 2 métodos de optimización (existentes en Tensorflow), ajústalos y compáralos. ¿Cual de los métodos te gusta más y porqué preferirías uno sobre los otros?

4. ¿Que pasa si cambias el tamaño de los minibatches?

5. ¿Como harías si dejaste a medias un proceso de aprendizaje (en 10 epochs por ejemplo) y quisieras entrenar la red 10 epoch más, y mañana quisieras entrenarla otros 10 epoch más?

**Para contestar las preguntas, agrega cuantas celdas con comentarios y con códgo sean necesarias.** Aprovecha que las libretas de *Jupyter* te permite hacerte una especie de tutorial personalizado.

## Respuestas 1 y 5:
En la siguiente celda respondo la pregunta 1 y la pregunta 5.
Si aumentamos los epochs hasta cierto punto, los costos suelen ser peores que en anteriores. Yo pienso que cuando empieza a incrementar mucho de un epoch al siguiente es porque ya es innecesario. A la pregunta 5, pues simplemente restauras la sesión y a seguirle, como se ve en la siguiente celda donde le agrego 5 epochs mas.

In [17]:
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, archivo_modelo)
    for epoch in range(5):

        #  Inicializa el costo promedio de todos los minibatches en 0
        avg_cost = 0.
        
        #  Calcula el número de minibatches que se pueden usar 
        total_batch = int(mnist.train.num_examples/tamano_minibatch)

        #  Por cada minibatch
        for i in range(total_batch):
            
            #  Utiliza un generador incluido en mnist que obtiene 
            #  tamano_minibatch ejemplos selecionados aleatoriamente del total
            batch_x, batch_y = mnist.train.next_batch(tamano_minibatch)
            
            #  Ejecuta la ops del paso_entrenamiento para aprender 
            #  y la del costo, con el fin de mostrar el aprendizaje
            _, c = sess.run([paso_entrenamiento, costo], feed_dict={x: batch_x, y: batch_y})
            
            #  Calcula el costo del minibatch y lo agrega al costo total
            avg_cost += c / total_batch
        
        # Muestra los resultados
        if epoch % display_step == 0:
            print (("Epoch: " + str(epoch)).ljust(20)
                   + ("Costo: " + str(avg_cost)))
    
    #  Guarda la sesión en el archivo rnn2.cptk
    saver.save(sess, archivo_modelo)

INFO:tensorflow:Restoring parameters from rnn2.ckpt
Epoch: 0            Costo: 0.22585296401817503
Epoch: 1            Costo: 0.27072849035227037
Epoch: 2            Costo: 0.21022210037467653
Epoch: 3            Costo: 0.3171673910534278
Epoch: 4            Costo: 0.2602260868140038


## Respuesta 2: cambiar la tasa de aprendizaje.

En las siguientes celdas cambio $\alpha$ de 0.001 a 0.005

In [19]:
alfa = 0.005
optimizador = tf.train.AdamOptimizer(learning_rate=alfa)
paso_entrenamiento = optimizador.minimize(costo)
archivo_modelo = "rnn2_2.ckpt"
saver = tf.train.Saver()

numero_epochs = 30
tamano_minibatch = 100
display_step = 1

# Muy importante la primera vez que se ejecuta inicializar todas las variables
init = tf.global_variables_initializer()

# La manera correcta de iniciar una sesión y realizar calculos
with tf.Session() as sess:
    sess.run(init)

    # Ciclos de entrenamiento
    for epoch in range(numero_epochs):

        #  Inicializa el costo promedio de todos los minibatches en 0
        avg_cost = 0.
        
        #  Calcula el número de minibatches que se pueden usar 
        total_batch = int(mnist.train.num_examples/tamano_minibatch)

        #  Por cada minibatch
        for i in range(total_batch):
            
            #  Utiliza un generador incluido en mnist que obtiene 
            #  tamano_minibatch ejemplos selecionados aleatoriamente del total
            batch_x, batch_y = mnist.train.next_batch(tamano_minibatch)
            
            #  Ejecuta la ops del paso_entrenamiento para aprender 
            #  y la del costo, con el fin de mostrar el aprendizaje
            _, c = sess.run([paso_entrenamiento, costo], feed_dict={x: batch_x, y: batch_y})
            
            #  Calcula el costo del minibatch y lo agrega al costo total
            avg_cost += c / total_batch
        
        # Muestra los resultados
        if epoch % display_step == 0:
            print (("Epoch: " + str(epoch)).ljust(20)
                   + ("Costo: " + str(avg_cost)))
    
    #  Guarda la sesión en el archivo rnn2.cptk
    saver.save(sess, archivo_modelo)
    
    print("Se acabaron los epochs, saliendo de la sesión de tensorflow.")

Epoch: 0            Costo: 67.42453009822154
Epoch: 1            Costo: 14.028810391100969
Epoch: 2            Costo: 7.092596803226988
Epoch: 3            Costo: 4.379405946387617
Epoch: 4            Costo: 3.072698492023819
Epoch: 5            Costo: 2.5449609771081194
Epoch: 6            Costo: 2.0939389536438333
Epoch: 7            Costo: 1.8440273851540918
Epoch: 8            Costo: 1.6476929244149332
Epoch: 9            Costo: 1.5661655998431794
Epoch: 10           Costo: 1.6305917877109648
Epoch: 11           Costo: 1.4128501150242117
Epoch: 12           Costo: 1.4433809464402723
Epoch: 13           Costo: 1.1167265752200881
Epoch: 14           Costo: 1.0598739839982623
Epoch: 15           Costo: 0.9879012744474929
Epoch: 16           Costo: 1.0215290464677098
Epoch: 17           Costo: 1.1047334009648508
Epoch: 18           Costo: 1.0995454032459238
Epoch: 19           Costo: 0.8798072780188029
Epoch: 20           Costo: 0.7830803893467078
Epoch: 21           Costo: 0.721785485

In [20]:
prediction_correcta = tf.equal(tf.argmax(estimado, 1), tf.argmax(y, 1))

precision = tf.reduce_mean(tf.cast(prediction_correcta, "float"))
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, archivo_modelo)
    porcentaje_acierto = sess.run(precision, feed_dict={x: mnist.test.images,
                                                        y: mnist.test.labels})
    print("Precisión: {}".format(porcentaje_acierto))

INFO:tensorflow:Restoring parameters from rnn2_2.ckpt
Precisión: 0.9671000242233276


### ¿Qué fue lo que ocurrió?

El costo de los últimos epochs es mayor, pero también la precisión es mayor que con la otra tasa.

## En las siguientes celdas voy a trabajar con una alfa menor.

In [21]:
alfa = 0.0005
optimizador = tf.train.AdamOptimizer(learning_rate=alfa)
paso_entrenamiento = optimizador.minimize(costo)
archivo_modelo = "rnn2_3.ckpt"
saver = tf.train.Saver()

numero_epochs = 30
tamano_minibatch = 100
display_step = 1

# Muy importante la primera vez que se ejecuta inicializar todas las variables
init = tf.global_variables_initializer()

# La manera correcta de iniciar una sesión y realizar calculos
with tf.Session() as sess:
    sess.run(init)

    # Ciclos de entrenamiento
    for epoch in range(numero_epochs):

        #  Inicializa el costo promedio de todos los minibatches en 0
        avg_cost = 0.
        
        #  Calcula el número de minibatches que se pueden usar 
        total_batch = int(mnist.train.num_examples/tamano_minibatch)

        #  Por cada minibatch
        for i in range(total_batch):
            
            #  Utiliza un generador incluido en mnist que obtiene 
            #  tamano_minibatch ejemplos selecionados aleatoriamente del total
            batch_x, batch_y = mnist.train.next_batch(tamano_minibatch)
            
            #  Ejecuta la ops del paso_entrenamiento para aprender 
            #  y la del costo, con el fin de mostrar el aprendizaje
            _, c = sess.run([paso_entrenamiento, costo], feed_dict={x: batch_x, y: batch_y})
            
            #  Calcula el costo del minibatch y lo agrega al costo total
            avg_cost += c / total_batch
        
        # Muestra los resultados
        if epoch % display_step == 0:
            print (("Epoch: " + str(epoch)).ljust(20)
                   + ("Costo: " + str(avg_cost)))
    
    #  Guarda la sesión en el archivo rnn2.cptk
    saver.save(sess, archivo_modelo)
    
    print("Se acabaron los epochs, saliendo de la sesión de tensorflow.")

Epoch: 0            Costo: 268.8982536315916
Epoch: 1            Costo: 67.732899241014
Epoch: 2            Costo: 44.36489459969782
Epoch: 3            Costo: 32.928652651743434
Epoch: 4            Costo: 25.572606104503976
Epoch: 5            Costo: 20.52394082741304
Epoch: 6            Costo: 16.789277559735563
Epoch: 7            Costo: 13.65965102869678
Epoch: 8            Costo: 11.191044363205734
Epoch: 9            Costo: 9.180577590051897
Epoch: 10           Costo: 7.531853813818575
Epoch: 11           Costo: 6.094321851352283
Epoch: 12           Costo: 5.053000407742359
Epoch: 13           Costo: 4.099136940840116
Epoch: 14           Costo: 3.2777030786991816
Epoch: 15           Costo: 2.7402724098819564
Epoch: 16           Costo: 2.177062466288401
Epoch: 17           Costo: 1.7975061264388
Epoch: 18           Costo: 1.4553161403464967
Epoch: 19           Costo: 1.163153212487788
Epoch: 20           Costo: 0.9360187679747453
Epoch: 21           Costo: 0.7556056301544316
Epoch

In [24]:
prediction_correcta = tf.equal(tf.argmax(estimado, 1), tf.argmax(y, 1))

precision = tf.reduce_mean(tf.cast(prediction_correcta, "float"))
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, archivo_modelo)
    porcentaje_acierto = sess.run(precision, feed_dict={x: mnist.test.images,
                                                        y: mnist.test.labels})
    print("Precisión: {}".format(porcentaje_acierto))

INFO:tensorflow:Restoring parameters from rnn2_3.ckpt
Precisión: 0.9416999816894531


### ¿Qué ocurrió?

El costo es menor que en los otros dos casos, pero la precisión es menor en estos también.

# En las siguientes celdas vamos a tratar de profundizar en la tercer pregunta, con AdaGrad y AdaDelta, donde podremos ver que no son tan buenos como Adam

In [25]:
alfa = 0.001
optimizador = tf.train.AdadeltaOptimizer(learning_rate=alfa)
paso_entrenamiento = optimizador.minimize(costo)
archivo_modelo = "rnn2_4.ckpt"
saver = tf.train.Saver()

numero_epochs = 30
tamano_minibatch = 100
display_step = 1

# Muy importante la primera vez que se ejecuta inicializar todas las variables
init = tf.global_variables_initializer()

# La manera correcta de iniciar una sesión y realizar calculos
with tf.Session() as sess:
    sess.run(init)

    # Ciclos de entrenamiento
    for epoch in range(numero_epochs):

        #  Inicializa el costo promedio de todos los minibatches en 0
        avg_cost = 0.
        
        #  Calcula el número de minibatches que se pueden usar 
        total_batch = int(mnist.train.num_examples/tamano_minibatch)

        #  Por cada minibatch
        for i in range(total_batch):
            
            #  Utiliza un generador incluido en mnist que obtiene 
            #  tamano_minibatch ejemplos selecionados aleatoriamente del total
            batch_x, batch_y = mnist.train.next_batch(tamano_minibatch)
            
            #  Ejecuta la ops del paso_entrenamiento para aprender 
            #  y la del costo, con el fin de mostrar el aprendizaje
            _, c = sess.run([paso_entrenamiento, costo], feed_dict={x: batch_x, y: batch_y})
            
            #  Calcula el costo del minibatch y lo agrega al costo total
            avg_cost += c / total_batch
        
        # Muestra los resultados
        if epoch % display_step == 0:
            print (("Epoch: " + str(epoch)).ljust(20)
                   + ("Costo: " + str(avg_cost)))
    
    #  Guarda la sesión en el archivo rnn2.cptk
    saver.save(sess, archivo_modelo)
    
    print("Se acabaron los epochs, saliendo de la sesión de tensorflow.")

Epoch: 0            Costo: 2072.5828378018487
Epoch: 1            Costo: 2046.871650834518
Epoch: 2            Costo: 2020.024997114704
Epoch: 3            Costo: 1992.78256303267
Epoch: 4            Costo: 1965.5480217950994
Epoch: 5            Costo: 1938.5429492187488
Epoch: 6            Costo: 1911.9091088867183
Epoch: 7            Costo: 1885.6192280717335
Epoch: 8            Costo: 1859.8193386008516
Epoch: 9            Costo: 1834.5408498313184
Epoch: 10           Costo: 1809.6975929953833
Epoch: 11           Costo: 1785.3171357865779
Epoch: 12           Costo: 1761.416823286577
Epoch: 13           Costo: 1737.9962673117896
Epoch: 14           Costo: 1715.0270507812497
Epoch: 15           Costo: 1692.4275310724438
Epoch: 16           Costo: 1670.175143599077
Epoch: 17           Costo: 1648.3671823952425
Epoch: 18           Costo: 1627.0285211736505
Epoch: 19           Costo: 1606.148374023437
Epoch: 20           Costo: 1585.6470430131376
Epoch: 21           Costo: 1565.493505637

In [26]:
prediction_correcta = tf.equal(tf.argmax(estimado, 1), tf.argmax(y, 1))

precision = tf.reduce_mean(tf.cast(prediction_correcta, "float"))
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, archivo_modelo)
    porcentaje_acierto = sess.run(precision, feed_dict={x: mnist.test.images,
                                                        y: mnist.test.labels})
    print("Precisión: {}".format(porcentaje_acierto))

INFO:tensorflow:Restoring parameters from rnn2_4.ckpt
Precisión: 0.10450000315904617


In [28]:
alfa = 0.001
optimizador = tf.train.AdagradOptimizer(learning_rate=alfa)
paso_entrenamiento = optimizador.minimize(costo)
archivo_modelo = "rnn2_5.ckpt"
saver = tf.train.Saver()

numero_epochs = 30
tamano_minibatch = 100
display_step = 1

# Muy importante la primera vez que se ejecuta inicializar todas las variables
init = tf.global_variables_initializer()

# La manera correcta de iniciar una sesión y realizar calculos
with tf.Session() as sess:
    sess.run(init)

    # Ciclos de entrenamiento
    for epoch in range(numero_epochs):

        #  Inicializa el costo promedio de todos los minibatches en 0
        avg_cost = 0.
        
        #  Calcula el número de minibatches que se pueden usar 
        total_batch = int(mnist.train.num_examples/tamano_minibatch)

        #  Por cada minibatch
        for i in range(total_batch):
            
            #  Utiliza un generador incluido en mnist que obtiene 
            #  tamano_minibatch ejemplos selecionados aleatoriamente del total
            batch_x, batch_y = mnist.train.next_batch(tamano_minibatch)
            
            #  Ejecuta la ops del paso_entrenamiento para aprender 
            #  y la del costo, con el fin de mostrar el aprendizaje
            _, c = sess.run([paso_entrenamiento, costo], feed_dict={x: batch_x, y: batch_y})
            
            #  Calcula el costo del minibatch y lo agrega al costo total
            avg_cost += c / total_batch
        
        # Muestra los resultados
        if epoch % display_step == 0:
            print (("Epoch: " + str(epoch)).ljust(20)
                   + ("Costo: " + str(avg_cost)))
    
    #  Guarda la sesión en el archivo rnn2.cptk
    saver.save(sess, archivo_modelo)
    
    print("Se acabaron los epochs, saliendo de la sesión de tensorflow.")

Epoch: 0            Costo: 543.5082359730118
Epoch: 1            Costo: 302.8070646528764
Epoch: 2            Costo: 238.24787083018933
Epoch: 3            Costo: 204.21240298184475
Epoch: 4            Costo: 182.30707300359563
Epoch: 5            Costo: 166.7696078352493
Epoch: 6            Costo: 154.92452954378993
Epoch: 7            Costo: 145.53912606672824
Epoch: 8            Costo: 137.86781581531866
Epoch: 9            Costo: 131.41601968245047
Epoch: 10           Costo: 125.91215150312941
Epoch: 11           Costo: 121.10685810089102
Epoch: 12           Costo: 116.86915109807796
Epoch: 13           Costo: 113.11860066500574
Epoch: 14           Costo: 109.77313910050823
Epoch: 15           Costo: 106.73151474692614
Epoch: 16           Costo: 103.9614776125821
Epoch: 17           Costo: 101.42782879916095
Epoch: 18           Costo: 99.10496138832782
Epoch: 19           Costo: 96.95376964222292
Epoch: 20           Costo: 94.95854930530886
Epoch: 21           Costo: 93.09986308011

In [29]:
prediction_correcta = tf.equal(tf.argmax(estimado, 1), tf.argmax(y, 1))

precision = tf.reduce_mean(tf.cast(prediction_correcta, "float"))
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, archivo_modelo)
    porcentaje_acierto = sess.run(precision, feed_dict={x: mnist.test.images,
                                                        y: mnist.test.labels})
    print("Precisión: {}".format(porcentaje_acierto))

INFO:tensorflow:Restoring parameters from rnn2_5.ckpt
Precisión: 0.8109999895095825


## Por último, la pregunta 4:

In [30]:
alfa = 0.001
optimizador = tf.train.AdamOptimizer(learning_rate=alfa)
paso_entrenamiento = optimizador.minimize(costo)
archivo_modelo = "rnn2_6.ckpt"
saver = tf.train.Saver()

numero_epochs = 30
tamano_minibatch = 200
display_step = 1

# Muy importante la primera vez que se ejecuta inicializar todas las variables
init = tf.global_variables_initializer()

# La manera correcta de iniciar una sesión y realizar calculos
with tf.Session() as sess:
    sess.run(init)

    # Ciclos de entrenamiento
    for epoch in range(numero_epochs):

        #  Inicializa el costo promedio de todos los minibatches en 0
        avg_cost = 0.
        
        #  Calcula el número de minibatches que se pueden usar 
        total_batch = int(mnist.train.num_examples/tamano_minibatch)

        #  Por cada minibatch
        for i in range(total_batch):
            
            #  Utiliza un generador incluido en mnist que obtiene 
            #  tamano_minibatch ejemplos selecionados aleatoriamente del total
            batch_x, batch_y = mnist.train.next_batch(tamano_minibatch)
            
            #  Ejecuta la ops del paso_entrenamiento para aprender 
            #  y la del costo, con el fin de mostrar el aprendizaje
            _, c = sess.run([paso_entrenamiento, costo], feed_dict={x: batch_x, y: batch_y})
            
            #  Calcula el costo del minibatch y lo agrega al costo total
            avg_cost += c / total_batch
        
        # Muestra los resultados
        if epoch % display_step == 0:
            print (("Epoch: " + str(epoch)).ljust(20)
                   + ("Costo: " + str(avg_cost)))
    
    #  Guarda la sesión en el archivo rnn2.cptk
    saver.save(sess, archivo_modelo)
    
    print("Se acabaron los epochs, saliendo de la sesión de tensorflow.")

Epoch: 0            Costo: 238.85401790272138
Epoch: 1            Costo: 56.021324837424494
Epoch: 2            Costo: 36.61117274197667
Epoch: 3            Costo: 26.93925780729814
Epoch: 4            Costo: 20.725648827986273
Epoch: 5            Costo: 16.496593103408802
Epoch: 6            Costo: 13.193000920469105
Epoch: 7            Costo: 10.628852546865287
Epoch: 8            Costo: 8.64317891427061
Epoch: 9            Costo: 6.979011908802084
Epoch: 10           Costo: 5.628013364184987
Epoch: 11           Costo: 4.512750429131769
Epoch: 12           Costo: 3.6435479563174615
Epoch: 13           Costo: 2.878919887298889
Epoch: 14           Costo: 2.2625978013206622
Epoch: 15           Costo: 1.885035001711497
Epoch: 16           Costo: 1.4578745947457845
Epoch: 17           Costo: 1.171947878360165
Epoch: 18           Costo: 0.9433802358450388
Epoch: 19           Costo: 0.7571304102939177
Epoch: 20           Costo: 0.5801896355787027
Epoch: 21           Costo: 0.477989457421810

In [31]:
prediction_correcta = tf.equal(tf.argmax(estimado, 1), tf.argmax(y, 1))

precision = tf.reduce_mean(tf.cast(prediction_correcta, "float"))
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, archivo_modelo)
    porcentaje_acierto = sess.run(precision, feed_dict={x: mnist.test.images,
                                                        y: mnist.test.labels})
    print("Precisión: {}".format(porcentaje_acierto))

INFO:tensorflow:Restoring parameters from rnn2_6.ckpt
Precisión: 0.9473999738693237


In [32]:
alfa = 0.001
optimizador = tf.train.AdamOptimizer(learning_rate=alfa)
paso_entrenamiento = optimizador.minimize(costo)
archivo_modelo = "rnn2_7.ckpt"
saver = tf.train.Saver()

numero_epochs = 30
tamano_minibatch = 50
display_step = 1

# Muy importante la primera vez que se ejecuta inicializar todas las variables
init = tf.global_variables_initializer()

# La manera correcta de iniciar una sesión y realizar calculos
with tf.Session() as sess:
    sess.run(init)

    # Ciclos de entrenamiento
    for epoch in range(numero_epochs):

        #  Inicializa el costo promedio de todos los minibatches en 0
        avg_cost = 0.
        
        #  Calcula el número de minibatches que se pueden usar 
        total_batch = int(mnist.train.num_examples/tamano_minibatch)

        #  Por cada minibatch
        for i in range(total_batch):
            
            #  Utiliza un generador incluido en mnist que obtiene 
            #  tamano_minibatch ejemplos selecionados aleatoriamente del total
            batch_x, batch_y = mnist.train.next_batch(tamano_minibatch)
            
            #  Ejecuta la ops del paso_entrenamiento para aprender 
            #  y la del costo, con el fin de mostrar el aprendizaje
            _, c = sess.run([paso_entrenamiento, costo], feed_dict={x: batch_x, y: batch_y})
            
            #  Calcula el costo del minibatch y lo agrega al costo total
            avg_cost += c / total_batch
        
        # Muestra los resultados
        if epoch % display_step == 0:
            print (("Epoch: " + str(epoch)).ljust(20)
                   + ("Costo: " + str(avg_cost)))
    
    #  Guarda la sesión en el archivo rnn2.cptk
    saver.save(sess, archivo_modelo)
    
    print("Se acabaron los epochs, saliendo de la sesión de tensorflow.")

Epoch: 0            Costo: 120.67941065268082
Epoch: 1            Costo: 27.64880454273529
Epoch: 2            Costo: 16.102737513962797
Epoch: 3            Costo: 10.42797812332574
Epoch: 4            Costo: 7.062980440349369
Epoch: 5            Costo: 4.773725820456125
Epoch: 6            Costo: 3.307046400818095
Epoch: 7            Costo: 2.3223761611578118
Epoch: 8            Costo: 1.8195204557146345
Epoch: 9            Costo: 1.3712785482127825
Epoch: 10           Costo: 1.0636963049033246
Epoch: 11           Costo: 0.8807433100314411
Epoch: 12           Costo: 0.771544613740846
Epoch: 13           Costo: 0.6757472381915435
Epoch: 14           Costo: 0.5726160679080549
Epoch: 15           Costo: 0.5383736161149612
Epoch: 16           Costo: 0.4340911798406418
Epoch: 17           Costo: 0.4656381717967164
Epoch: 18           Costo: 0.3820271855458882
Epoch: 19           Costo: 0.4523016800839315
Epoch: 20           Costo: 0.4112177723228154
Epoch: 21           Costo: 0.34313236785

In [33]:
prediction_correcta = tf.equal(tf.argmax(estimado, 1), tf.argmax(y, 1))

precision = tf.reduce_mean(tf.cast(prediction_correcta, "float"))
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, archivo_modelo)
    porcentaje_acierto = sess.run(precision, feed_dict={x: mnist.test.images,
                                                        y: mnist.test.labels})
    print("Precisión: {}".format(porcentaje_acierto))

INFO:tensorflow:Restoring parameters from rnn2_7.ckpt
Precisión: 0.9628000259399414


# Como se pudo ver...

el incrementar el doble del tamaño en los minibatches dio lugar a una menor precisión, mientras que al dividirla a la mitad dio una mejor precisión con el mismo optimizador, función de costo y $\alpha$